**Prj04**

Consider the following data sets:

- Today: Sep 27, 2018

- Spot price: 290.68

- Maturity: Dec 31, 2018

- Strike: 288

- rate: 0.02 (from https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield)

- Market call price: 9.23

**Todo**

1. If the volatility is $30\%$, what is the difference between BSM call price and market call price?
2. Compute the implied volatility.

* ***BSM_option_valuation***

In [1]:
import numpy as np
import scipy.stats as ss
import time 
import math

def d1f(St, K, t, T, r, sigma):
    ''' Black-Scholes-Merton d1 function.
        Parameters see e.g. BSM_call_value function. '''
    d1 = (math.log(St / K) + (r + 0.5 * sigma ** 2)
          * (T - t)) / (sigma * math.sqrt(T - t))
    return d1

def BSM_call_value(St, K, t, T, r, sigma):
    ''' Calculates Black-Scholes-Merton European call option value.

    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    sigma : float
        volatility

    Returns
    =======
    call_value : float
        European call present value at t
    '''
    d1 = d1f(St, K, t, T, r, sigma)
    d2 = d1 - sigma * math.sqrt(T - t)
    call_value = St * ss.norm.cdf(d1) - math.exp(-r * (T - t)) * K * ss.norm.cdf(d2)
    return call_value

def BSM_put_value(St, K, t, T, r, sigma):
    ''' Calculates Black-Scholes-Merton European put option value.

    Parameters
    ==========
    St : float
        stock/index level at time t
    K : float
        strike price
    t : float
        valuation date
    T : float
        date of maturity/time-to-maturity if t = 0; T > t
    r : float
        constant, risk-less short rate
    sigma : float
        volatility

    Returns
    =======
    put_value : float
        European put present value at t
    '''
    put_value = BSM_call_value(St, K, t, T, r, sigma) \
        - St + math.exp(-r * (T - t)) * K
    return put_value

# Solution to Q1
**the difference between BSM call price and market call price**

In [2]:
import pandas as pd
import datetime
Maturity = datetime.datetime(2018,12,31)
Today = datetime.datetime(2018,9,27)
T = (Maturity - Today).days / 365.
print('Maturity: %s \n'%(Maturity),'Today: %s \n'%(Today), 'T: %s'%(T))

Maturity: 2018-12-31 00:00:00 
 Today: 2018-09-27 00:00:00 
 T: 0.2602739726027397


In [3]:
S0 = 290.68
K = 288
r = 0.02
sigma = 0.3
call_marketprice = 9.23
diff = BSM_call_value(S0, K, 0, T, r, sigma) - call_marketprice
if diff > 0 :
    print ('BSM_call_value is $%s more expensive than call_marketprice'%(diff))
else :
    print ('call_marketprice $%s more expensive than BSM_call_value'%(diff))

BSM_call_value is $10.539937820552115 more expensive than call_marketprice


# Solution to Q2
**Compute the implied volatility**

In [8]:
from scipy import optimize
def IVolBsm(S0, K, T, r, P0):
    """
    Inputs:
        S0: spot price
        K: strike
        T: time to maturity in year
        r: rate
        P0: market price
    Outputs:
        Implied volatility
    """
    InitVol = .3
    error = lambda sigma: (BSM_call_value(S0, K, 0, T, r, sigma) - P0)**2
    opt = optimize.fmin(error, InitVol);
    return opt[0]

In [9]:
print('Implied volatility is %f \n' %IVolBsm(S0, K, T, r, call_marketprice),
      'Now the BSM_call_value with implied volatility is : %f'%(BSM_call_value(S0, K, 0, T, r, IVolBsm(S0, K, T, r, call_marketprice))))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 15
         Function evaluations: 30
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 15
         Function evaluations: 30
Implied volatility is 0.118242 
 Now the BSM_call_value with implied volatility is : 9.230463
